# Challenge 
Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are ***tasked with predicting their daily sales for up to six weeks in advance***. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In their first Kaggle competition, Rossmann is challenging you to predict 6 weeks of daily sales for 1,115 stores located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams! 

# Predict Store Sales 
***What we learn from this notebook***

1. **Data Collection and Preparation**
   * Gather Data
   * Remove **TOTALLY** unrelated columns from dataset
   * Encode cateorical features
   * Handle too many unique values in categorical features (Cordinality)
   * Missing value handling 
  
  
2. **Exploratory Data Analysis (EDA)**
   * Remove outliers
   * Understand data distribution (mean, median, mode, standard deviation).
   * Check for correlations & covariance between features.
   
   
3. **Feature Selection & Engineering**
   *  Select relevant features based on correlation, mutual information, feature importance, Lasso, RFE, R-squared, p-values, and VIF.
   * Create new features if necessary.
   
   
4. **Check for Assumptions of Linear Regression**
   * **Linearity**: The relationship between predictors and the target should be linear.
   * **Independence**: Observations should be independent of each other,
   * **Homoscedasticity**: Constant variance of the residuals
   * **Normality of Residuals**: Residuals should be normally distributed.
   * **No Multicollinearity**: Predictors should not be highly correlated with each other.
   
   
5. **Model Training**
   * Split data into training and testing sets.
   * Train the multiple linear regression model.
   
   
6. **Model Evaluation**
   * Evaluate the model using metrics like R-squared, adjusted R-squared, Mean Squared Error (MSE), and Root Mean Squared Error (RMSE).
   * Check residual plots to ensure assumptions are met.
   
   
7. **Predict Sales based on new model created**
 

## 1. Data Collection and Preparation

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        full_path = os.path.join(dirname, filename)
        print (full_path)
        if "store.csv" in full_path:
            df_store = pd.read_csv(full_path)
        elif "train.csv" in full_path:
            df_train = pd.read_csv(full_path)
        elif "test.csv" in full_path:
            df_test = pd.read_csv(full_path)
                                 
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rossmann-store-sales-dataset/store.csv
/kaggle/input/rossmann-store-sales-dataset/train.csv


/tmp/ipykernel_18/1702920036.py:19: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(full_path)


/kaggle/input/rossmann-store-sales-dataset/test.csv


### Data fields
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

**Id** - an Id that represents a (Store, Date) duple within the test set

**Store** - a unique Id for each store

**Sales** - the turnover for any given day (this is what you are predicting)

**Customers** - the number of customers on a given day

**Open** - an indicator for whether the store was open: 0 = closed, 1 = open

**StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None

**SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools

**StoreType** - differentiates between 4 different store models: a, b, c, d

**Assortment** - describes an assortment level: a = basic, b = extra, c = extended

**CompetitionDistance** - distance in meters to the nearest competitor store

**CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened

**Promo** - indicates whether a store is running a promo on that day

**Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating

**Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2

**PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


In [2]:
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [3]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
